In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


# Load and preprocess the data
#data = pd.read_csv('df14.csv')  # Replace 'your_data.csv' with the path to your dataset

import pandas as pd

for i in range(1, 15):
    filename = f"df{i}.csv"
    data = pd.read_csv(filename)
    data = data[['year-week', 'ID_material', 'Units']]
    data = data.pivot(index='year-week', columns='ID_material', values='Units')
    data = data.fillna(0)  # Replace NaN values with 0 or handle them based on your requirements

    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    # Generate input sequences and target values
    sequence_length = 10
    input_sequences = []
    target_values = []
    for i in range(len(scaled_data) - sequence_length - 16):  # Adjust -8 to match the desired prediction length
        input_sequences.append(scaled_data[i:i+sequence_length])
        target_values.append(scaled_data[i+sequence_length:i+sequence_length+8])

    input_sequences = np.array(input_sequences)
    target_values = np.array(target_values)

    # Split the data into training and validation sets
    split_ratio = 0.8  # 80% for training, 20% for validation
    split_index = int(split_ratio * len(input_sequences))

    x_train = input_sequences[:split_index]
    y_train = target_values[:split_index]
    x_val = input_sequences[split_index:]
    y_val = target_values[split_index:]

    # Build the RNN model
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(sequence_length, data.shape[1])))
    model.add(Dense(8))  # 8 is the length of the target values to be predicted

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

    # Make predictions on new data
    new_input_sequence = scaled_data[-sequence_length:]  # Use the last available sequence as input
    new_input_sequence = np.expand_dims(new_input_sequence, axis=0)
    predictions = model.predict(new_input_sequence)

    # Inverse scale the predictions
    predictions = scaler.inverse_transform(predictions)

    print("Predictions:")
    print(predictions)

    actual_values = data.iloc[-8:].values  # Assuming you have the actual values for the last 8 time steps
    given_array = actual_values
    # Convert shape to (1, 8)
    converted_array = given_array.transpose()  # or converted_array = given_array.T

    print(converted_array)
    print(converted_array.shape)

    # Calculate RMSE

    rmse = np.sqrt(mean_squared_error(converted_array, predictions))
    print("RMSE:", rmse)
    
    grouped_predictions = np.sum(predictions.reshape(-1, 4), axis=1)
    grouped_actual_values = np.sum(converted_array.reshape(-1, 4), axis=1)

    print("Grouped Predictions:")
    print(grouped_predictions)
    print("Grouped Actual Values:")
    print(grouped_actual_values)



Epoch 1/10
7/7 [==============================] - 2s 72ms/step - loss: 0.2169 - val_loss: 0.2621
Epoch 2/10
7/7 [==============================] - 0s 14ms/step - loss: 0.1820 - val_loss: 0.2149
Epoch 3/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1411 - val_loss: 0.1587
Epoch 4/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1032 - val_loss: 0.1329
Epoch 5/10
7/7 [==============================] - 0s 16ms/step - loss: 0.0927 - val_loss: 0.1161
Epoch 6/10
7/7 [==============================] - 0s 15ms/step - loss: 0.0824 - val_loss: 0.1138
Epoch 7/10
7/7 [==============================] - 0s 15ms/step - loss: 0.0811 - val_loss: 0.1116
Epoch 8/10
7/7 [==============================] - 0s 16ms/step - loss: 0.0807 - val_loss: 0.1114
Epoch 9/10
7/7 [==============================] - 0s 15ms/step - loss: 0.0803 - val_loss: 0.1114
Epoch 10/10
1/1 [==============================] - 0s 227ms/step
Predictions:
[[102.77275  102.55009  114.031845 106.22664  108

In [56]:
grouped_predictions = np.sum(predictions.reshape(-1, 4), axis=1)
grouped_actual_values = np.sum(converted_array.reshape(-1, 4), axis=1)

print("Grouped Predictions:")
print(grouped_predictions)
print("Grouped Actual Values:")
print(grouped_actual_values)



Grouped Predictions:
[5218.0303 5390.9756]
Grouped Actual Values:
[2604 4248]


In [57]:
predictions

array([[1351.5126, 1245.6602, 1257.1558, 1363.7015, 1398.709 , 1358.444 ,
        1479.4216, 1154.4011]], dtype=float32)

In [43]:
predictions.sum()

598.5833

In [44]:
converted_array.sum()

1920